## Compute permutations

This script reads the epoched data already imported from BVA, resampled, preprocessed and merged across subjects
and compute the both surrogate distribution of connectivity matrices and the true connectivity matrix.

No orthogonalization is used for surrogate distribution (which drastically increases the speed) because the signals are 
randomly shifted in time with respect to each other during each iteration

### Parameters to consider:
    numPerm = 500 - a number of permutations
    shiftEpch = 20 - a max number of epochs shift (measured in epochs, the overlap is not taken into acount)

### Inputs:
    pipePath  - path to a pipe folder containing epoch data
    fileName  - name of the epoch data (condition specific)
    
### Outputs:
    corrMat - the data correlation matrix (lower triangle)
    corrMatSurr - surrogate distribution of correlation matrices

### Define a working directory and load data
- define a number of conditions nC
- define a number of groups nG

In [8]:
import numpy as np

def init_list(dims, val):
    if len(dims) == 0:
        raise ValueError("Requires at least 1 dimension.")
    if len(dims) == 1:
        return [val for _ in range(dims[0])]
    return [init_list(dims[1:], val=val) for _ in range(dims[0])]


#Load the data:
pipePath = 'C:/FEL/ING/diplomka/data/TG/kamdu/'
nC = 3
nG = 1
# stare = 1, cross =0
epochData = init_list((nC,nG),val=0)

for cIdx in range(0,nC):
    for gIdx in range(0,nG):
        epochData[cIdx][gIdx] = np.load(pipePath + 'c' + str(cIdx) + 'g' + str(gIdx) + '.npz')["arr_0"] 
        

nEpoch, nChan, nSamp = epochData[1][0].shape
print('nEpochs: ' + str(nEpoch) + ' nChan: ' + str(nChan) + ' nSamp: ' + str(nSamp))

nEpochs: 13 nChan: 128 nSamp: 4097


[[[ 1.75797674e-05  1.92835537e-05  2.09252218e-05 ...  8.68613007e-06
    8.53789387e-06  8.36566973e-06]
  [ 1.40786886e-05  1.55648933e-05  1.70017386e-05 ...  9.89082912e-06
    9.72402248e-06  9.52875255e-06]
  [ 2.04162885e-05  2.21100869e-05  2.37304677e-05 ...  1.05638725e-05
    1.03721810e-05  1.01494756e-05]
  ...
  [-1.10031326e-06 -1.19179652e-06 -1.28084199e-06 ...  1.23263611e-06
    1.10724097e-06  9.77989050e-07]
  [-2.26703366e-06 -2.33587724e-06 -2.39592617e-06 ...  2.35285711e-06
    2.32490137e-06  2.29005291e-06]
  [-2.16911172e-06 -2.20508933e-06 -2.23288937e-06 ...  3.77957846e-07
    2.89598880e-07  1.99815892e-07]]

 [[-7.12438738e-07 -1.35597485e-07  4.39344311e-07 ...  3.50799889e-06
    3.05920662e-06  2.58876909e-06]
  [-5.12501175e-07 -2.18024139e-08  4.63914631e-07 ...  3.09241001e-06
    2.68797030e-06  2.26656823e-06]
  [-1.27420463e-06 -7.33857973e-07 -1.98336772e-07 ...  2.45498123e-06
    1.90125191e-06  1.33212883e-06]
  ...
  [ 1.56703045e-06  1.7

### Exclude or reduce some epochs if needed - skip 

In [21]:
decim = 4
epochData[0][0] = epochData[0][0][0:-1:decim,:,:]
newNEpoch = epochData[0][0].shape
newNEpoch

(472, 366, 128)

### Define functions

In [ ]:
def pairwise_correlation(A, B):
    am = A - np.mean(A, axis=0, keepdims=True)
    bm = B - np.mean(B, axis=0, keepdims=True)
    return am.T @ bm /  (np.sqrt(
        np.sum(am**2, axis=0,
               keepdims=True)).T * np.sqrt(
        np.sum(bm**2, axis=0, keepdims=True)))

def atin_powCorr_compute(epochsData):    
    nEpoch, nChan, nSamp = epochsData.shape
#prepare two arrays for orthogonalization and correlation
    x = np.zeros((nEpoch,nSamp))
    y = np.zeros((nEpoch,nSamp))
#prepare a nChanXnChan correlation matrix
    corrMat = np.zeros((nChan,nChan))
    for chan1Idx in range(0,nChan):
        for chan2Idx in range(0,chan1Idx):
#a pair of two channels and all Epochs
            x = epochsData[:,chan1Idx,:]
            y = epochsData[:,chan2Idx,:]
#across all epochs orthogonalize
            x = np.reshape(x, (nEpoch*nSamp,1),order='C')
            y = np.reshape(y, (nEpoch*nSamp,1),order='C')
#get the regression coeficient from the pseudoinverse
            beta = np.real(np.dot(np.linalg.pinv(x),y))
            y = y - beta*x
#rehape back to the epochXsamples matrix
            x = np.reshape(x, (nEpoch,nSamp),order='C')
            y = np.reshape(y, (nEpoch,nSamp),order='C')
#compute RMS
            xRMS = np.power(x,2)
            yRMS = np.power(y,2)
            xRMS = np.sqrt(np.mean(xRMS,axis=1)) # average over samples
            yRMS = np.sqrt(np.mean(yRMS,axis=1))
            corrMat[chan1Idx,chan2Idx] = pairwise_correlation(xRMS, yRMS) # for each pair of channels I have one sample for each epoch
    return corrMat

def atin_powCorr_compute_surr(epochsData):
    nEpoch, nChan = epochsData.shape
#prepare a nChanXnChan correlation matrix    
    corrMat = np.zeros((nChan,nChan))
    for chan1Idx in range(0,nChan):
        for chan2Idx in range(0,chan1Idx):            
#compute RMS
            xRMS = epochsData[:,chan1Idx]
            yRMS = epochsData[:,chan2Idx]
            corrMat[chan1Idx,chan2Idx] = pairwise_correlation(xRMS, yRMS)
    return corrMat          

### Compute the observed correlation matrix
- consider the lower triangular matrix is computed to save time
  

In [10]:
corrMat = init_list((nC,nG),val=0)
for cIdx, condIns in enumerate(epochData):
    for gIdx, groupIns in enumerate(condIns):
        corrMat[cIdx][gIdx] = atin_powCorr_compute(groupIns)

C:\Users\Martin\AppData\Local\Temp\ipykernel_19284\501741627.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  corrMat[chan1Idx,chan2Idx] = pairwise_correlation(xRMS, yRMS)


### and save the results...

In [11]:
for cIdx, condIns in enumerate(corrMat):
    for gIdx, groupIns in enumerate(condIns):
        np.save(pipePath + 'c' + str(cIdx) + 'g' + str(gIdx) + '_observed', groupIns)

### Slightly change the correlation matrix computation:
- since we do a random circular shifts in time we do not need to orthogonalize
- then we can compute RMS within each epoch before itarating across electrode pairs
- this will save us a lot of time and this is implemented in the **corrMatSurr** function

In [13]:
from tqdm import trange
numPerm = 200
shiftEpch = 20

#prepare RMS (the computationally demanding step)
rmsMat = init_list((nC,nG),val=0)
for cIdx in range(0,nC):
    for gIdx in range(0,nG):
        rmsMat[cIdx][gIdx] =  np.sqrt(np.mean(np.power(epochData[cIdx][gIdx],2),2))



corrMatSurr = init_list((nC,nG),val=0)

for cIdx, condIns in enumerate(rmsMat):
    for gIdx, groupIns in enumerate(condIns):
        npCorrMat = []
        print('cond: ' + str(cIdx) + ' group: ' + str(gIdx) + ' permuting...')
        for iterIdx in trange(numPerm):            
            epochShift = np.copy(groupIns)
            for chanIdx in range(0,nChan): 
                randShift = np.random.randint(-shiftEpch,shiftEpch,1)
                epochShift[:,chanIdx] = np.roll(epochShift[:,chanIdx],randShift,axis=0)
            npCorrMat.append(atin_powCorr_compute_surr(epochShift))
        corrMatSurr[cIdx][gIdx] = np.array(npCorrMat)

for cIdx, condIns in enumerate(corrMatSurr):
    for gIdx, groupIns in enumerate(condIns):
        np.save(pipePath + 'c' + str(cIdx) + 'g' + str(gIdx) + '_surrogate', groupIns)

cond: 0 group: 0 permuting...


  0%|          | 0/200 [00:00<?, ?it/s]C:\Users\Martin\AppData\Local\Temp\ipykernel_19284\501741627.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  corrMat[chan1Idx,chan2Idx] = pairwise_correlation(xRMS, yRMS)
100%|██████████| 200/200 [01:58<00:00,  1.69it/s]


cond: 1 group: 0 permuting...


100%|██████████| 200/200 [01:59<00:00,  1.67it/s]


cond: 2 group: 0 permuting...


100%|██████████| 200/200 [01:57<00:00,  1.70it/s]


### and save the results again...

In [14]:
for cIdx, condIns in enumerate(corrMatSurr):
    for gIdx, groupIns in enumerate(condIns):
        np.save(pipePath + 'c' + str(cIdx) + 'g' + str(gIdx) + '_surrogate', groupIns)